In [1]:
import uproot
import awkward
import numpy as np
from uproot_methods import TLorentzVectorArray

from coffea.processor import LazyDataFrame
from coffea.analysis_objects import JaggedCandidateArray


In [2]:
#fn = 'root://cmsxrootd.fnal.gov//store/mc/RunIIAutumn18NanoAODv6/ZZZ_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/Nano25Oct2019_102X_upgrade2018_realistic_v20_ext1-v1/40000/0797B437-5B3E-1943-9F3D-9E84D64D1275.root'
## uproot xrootd doesn't work in my environment
fn = '/hadoop/cms/store/user/dspitzba/nanoAOD/ttw_samples/0p1p5/ZZZ_TuneCP5_13TeV-amcatnlo-pythia8__RunIIAutumn18NanoAODv6-Nano25Oct2019_102X_upgrade2018_realistic_v20_ext1-v1/ZZZ_TuneCP5_13TeV-amcatnlo-pythia8__RunIIAutumn18NanoAODv6-Nano25Oct2019_102X_upgrade2018_realistic_v20_ext1-v1_1.root'

file = uproot.open(fn)

In [25]:
tree = file['Events']
df = LazyDataFrame(tree, flatten=True)

jets = JaggedCandidateArray.candidatesfromcounts(
    df['nJet'],
    pt=df['Jet_pt'],
    eta=df['Jet_eta'],
    phi=df['Jet_phi'],
    mass=df['Jet_mass'],
    btag=df['Jet_btagDeepB'],
    #ctag=df['Jet_btagDeepC'],
)
muons = JaggedCandidateArray.candidatesfromcounts(
    df['nMuon'],
    pt=df['Muon_pt'],
    eta=df['Muon_eta'],
    phi=df['Muon_phi'],
    mass= 0 * df['Muon_pt'],
)

In [5]:
# example for getting events with at least one jet with pt>100 GeV
jets[(jets.pt>100).sum()>0].pt

<JaggedArray [[143.0 71.9375 48.34375 41.0625 18.53125 17.84375] [193.5 149.75 63.09375 ... 28.40625 20.40625 16.5625] [118.75 58.5625 42.5 ... 23.34375 22.1875 15.203125] ... [176.5 150.125 94.4375 ... 21.828125 18.21875 16.71875] [366.75 289.75 169.25 ... 18.09375 17.21875 16.875] [201.125 75.125 37.5 33.96875 15.203125]] at 0x7f26905c2cf8>

In [38]:
bjets = jets[(jets['btag']>0.8)] # random selection for b-tags
lightjets = jets[(jets['btag']<0.9)] # not exactly anti-selecting jets so to have an overlap
leadingbjet = bjets[bjets.pt.argmax()]

/home/users/dspitzba/TTW/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1035: RuntimeWarning: invalid value encountered in greater
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/dspitzba/TTW/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1035: RuntimeWarning: invalid value encountered in less
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [48]:
lightjets[~lightjets.match(leadingbjet, deltaRCut=0.4)].pt

<JaggedArray [[143.0 71.9375 41.0625 18.53125 17.84375] [193.5 149.75 45.1875 29.875 28.40625 16.5625] [86.625 82.5625 62.46875 47.25] ... [150.125 94.4375 79.8125 ... 21.828125 18.21875 16.71875] [366.75 289.75 169.25 ... 18.09375 17.21875 16.875] [201.125 75.125 33.96875 15.203125]] at 0x7f2690453278>

In [45]:
bjets.pt

<JaggedArray [[48.34375 41.0625] [63.09375 20.40625] [] ... [176.5] [] [37.5]] at 0x7f2690445fd0>

In [46]:
lightjets.pt

<JaggedArray [[143.0 71.9375 41.0625 18.53125 17.84375] [193.5 149.75 63.09375 ... 29.875 28.40625 16.5625] [86.625 82.5625 62.46875 47.25] ... [176.5 150.125 94.4375 ... 21.828125 18.21875 16.71875] [366.75 289.75 169.25 ... 18.09375 17.21875 16.875] [201.125 75.125 37.5 33.96875 15.203125]] at 0x7f26903c8e48>